In [ ]:
import pandas as pd
import tkinter as tk
import warnings
import os
from tkinter import messagebox
from datetime import datetime
import calendar

warnings.filterwarnings('ignore')

def consolidar_archivos(agencia_path: str, archivos: list) -> pd.DataFrame:
    dataframes = []
    for archivo in archivos:
        df = pd.read_excel(os.path.join(agencia_path, archivo), dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str})
        dataframes.append(df)
    
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    df_consolidado.to_excel(os.path.join(agencia_path, f'CONSOLIDADO_GESTIONES_{agencia_path.split('/')[-1]}.xlsx'), index = False)
    return df_consolidado

rj_path = 'bases/gestiones/RJ'
clasa_path = 'bases/gestiones/CLASA'
mornese_path = 'bases/gestiones/MORNESE'

rj_archivos = [file for file in os.listdir(rj_path) if file.startswith('REP_GESTIONES _ BBVA') and file.endswith('.xlsx')]
clasa_archivos = [file for file in os.listdir(clasa_path) if file.startswith('REP_GESTIONES _ BBVA') and file.endswith('.xlsx')]
mornese_archivos = [file for file in os.listdir(mornese_path) if file.startswith('REP_GESTIONES _ BBVA') and file.endswith('.xlsx')]

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar 3 meses RJ?', icon='warning')
if result == 'yes':
    path_rj_ = f'{rj_path}/Cortes de gestion RJ Sep,Oct al 13.11.xlsx'
    df_rj_ = pd.read_excel(path_rj_, dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str})
    df_rj_.drop(columns=['Hora'], inplace=True)
    print('RJ Sep,Oct,Nov:', df_rj_.shape)
    
    df_09 = df_rj_[(df_rj_['TIPO_CONTACTO'] == 'CONTACTO DIRECTO') & (df_rj_['FECHA_LLAMADA'].dt.month == 9)]
    df_09.to_excel(f'{rj_path}/REP_GESTIONES _ BBVA _0924.xlsx', index=False)
    
    df_10 = df_rj_[(df_rj_['TIPO_CONTACTO'] == 'CONTACTO DIRECTO') & (df_rj_['FECHA_LLAMADA'].dt.month == 10)]
    df_10.to_excel(f'{rj_path}/REP_GESTIONES _ BBVA _1024.xlsx', index=False)
    
    df_11 = df_rj_[(df_rj_['TIPO_CONTACTO'] == 'CONTACTO DIRECTO') & (df_rj_['FECHA_LLAMADA'].dt.month == 11)]
    df_11.to_excel(f'{rj_path}/REP_GESTIONES _ BBVA _1124.xlsx', index=False)
    
    print('RJ Sep,Oct,Nov Contacto Directo:', df_09.shape[0] + df_10.shape[0] + df_11.shape[0])

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Consolidar archivos de gestiones?', icon='warning')
if result == 'yes':
    df_rj = consolidar_archivos(rj_path, rj_archivos)
    print('Consolidado RJ:', df_rj.shape)
    
    #df_clasa = consolidar_archivos(clasa_path, clasa_archivos)
    #print('Consolidado CLASA:', df_clasa.shape)
    
    #df_mornese = consolidar_archivos(mornese_path, mornese_archivos)
    #print('Consolidado Mornese:', df_mornese.shape)

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Leer consolidados?', icon='warning')
if result == 'yes':
    df_rj = pd.read_excel(f'{rj_path}/CONSOLIDADO_GESTIONES_RJ.xlsx', dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str})
    #df_clasa = pd.read_excel(f'{clasa_path}/CONSOLIDADO_GESTIONES_CLASA.xlsx', dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str})
    #df_mornese = pd.read_excel(f'{mornese_path}/CONSOLIDADO_GESTIONES_MORNESE.xlsx', dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str})

root.destroy()

In [ ]:
df_rj_test = df_rj.copy()
#df_clasa_test = df_clasa.copy()
#df_mornese_test = df_mornese.copy()

In [ ]:
df_rj_test.columns

In [ ]:
df_rj_test.dtypes

In [ ]:
df_rj_test['TIPO_CONTACTO'].value_counts(dropna=False)

In [ ]:
df_rj_test['FECHA_LLAMADA'].dt.month.value_counts(dropna=False)

In [ ]:
df_rj_test.isna().sum()

In [ ]:
df_rj_test['HORA_INICIO_LLAMADA']

In [ ]:
df_rj_test['HORA_INICIO_LLAMADA'] = pd.to_datetime(df_rj_test['HORA_INICIO_LLAMADA'])
df_rj_test['HORA_INICIO_LLAMADA_HORA'] = df_rj_test['HORA_INICIO_LLAMADA'].dt.hour
df_rj_test['HORA_INICIO_LLAMADA_HORA']

In [ ]:
df_rj_test['TRAMO'].value_counts(dropna=False)

In [ ]:
df_rj_test['PRODUCTO'].value_counts(dropna=False)

In [ ]:
df_rj_test['MARCA'] = df_rj_test['MARCA'].str.upper().replace(' ', '', regex=True)

In [ ]:
df_rj_test['MARCA'].value_counts(dropna=False)

In [ ]:
df_rj_test